# Weighted Shortest Paths 🚀

**Instructors:** Erik Demaine, Jason Ku, and Justin Solomon

> *"In the real world, not all paths are created equal. Sometimes the shortest route isn't about counting steps—it's about counting costs."*

---

## Table of Contents
1. [🔄 Quick Refresher: What We Know](#review)
2. [⚖️ Welcome to Weighted Graphs](#weighted-graphs)
3. [🛤️ The Art of Weighted Paths](#weighted-paths)
4. [🗺️ Your Shortest Path Toolkit](#algorithms-overview)
5. [🌳 Building Your Path Tree](#shortest-paths-tree)
6. [🎯 DAG Relaxation: The Magic Algorithm](#dag-relaxation)
7. [✅ Why It Actually Works](#correctness)
8. [⏱️ Speed Analysis](#running-time)

---

## 🔄 Quick Refresher: What We Know {#review}

Think of this as your algorithmic toolkit so far. You've been exploring graphs where every step costs the same—like walking through a city where every block takes exactly 1 minute.

### Your Current Arsenal:
- **Single-Source Shortest Paths with BFS**: $O(|V| + |E|)$ 
  - *"How many hops to get from A to B?"* (returns distance per vertex)
- **Single-Source Reachability with BFS or DFS**: $O(|E|)$ 
  - *"Can I even get there from here?"* (returns only reachable vertices)
- **Connected Components with Full-BFS or Full-DFS**: $O(|V| + |E|)$ 
  - *"What parts of this graph hang out together?"*
- **Topological Sort of DAG with Full-DFS**: $O(|V| + |E|)$ 
  - *"What order should I do things in?"* (Like getting dressed: socks before shoes!)

### 🎯 The Big Shift
- **Before**: Distance = counting steps (like "6 degrees of Kevin Bacon")
- **Today**: Distance = accumulated cost (like your GPS calculating fastest route considering traffic)

---

## ⚖️ Welcome to Weighted Graphs {#weighted-graphs}

### What's a Weighted Graph? 🤔

Imagine you're planning a road trip. Not every road segment is the same:
- Highway: 100 miles, 1.5 hours
- City streets: 5 miles, 30 minutes  
- Mountain pass: 20 miles, 2 hours

A **weighted graph** $G = (V, E)$ + weight function $w : E \to \mathbb{Z}$ captures this reality!

### 🌍 Real-World Weight Examples

| Application | What Weights Represent |
|-------------|----------------------|
| **🗺️ GPS Navigation** | Travel time, distance, or fuel cost |
| **💰 Financial Networks** | Transaction fees or exchange rates |
| **👥 Social Media** | How close friends are (comment frequency?) |
| **🌐 Internet Routing** | Latency, bandwidth, or reliability |
| **🎮 Game Paths** | Movement cost, danger level, or energy |

### 💡 Quick Example: The Coffee Shop Problem

You're at home (H) and want to visit three places:
- Coffee shop (C): 2 minutes away
- Friend's house (F): 15 minutes  
- Grocery store (G): 8 minutes

But there are shortcuts! F to G is only 3 minutes. Now finding the shortest path becomes interesting...

### 💾 How We Store This Computationally

**Option 1 - Embedded Style:**
```python
# Adjacency list with weights
graph = {
    'H': [('C', 2), ('F', 15), ('G', 8)],
    'C': [('F', 10)],
    'F': [('G', 3)],
    'G': []
}
```

**Option 2 - Separate Weight Map:**
```python
# Graph structure + separate weights
edges = [('H','C'), ('H','F'), ('H','G'), ('C','F'), ('F','G')]
weights = {('H','C'): 2, ('H','F'): 15, ('H','G'): 8, ('C','F'): 10, ('F','G'): 3}
```

**Key Requirement**: We need $O(1)$ weight lookups (no searching through lists!)

### 📊 Visual Examples
[**Image Placeholder**: Two weighted graphs G1 and G2 showing vertices a,b,c,d,e,f,g,h with various positive and negative edge weights. Include a legend explaining what negative weights might represent]
![G1 & G2](./01-Graphs.png)

---

## 🛤️ The Art of Weighted Paths {#weighted-paths}

### 💰 Path Weight = Your Total Bill

If you take a path $\pi$ through a weighted graph, your **total cost** is:
$$w(\pi) = \sum_{\text{edges in } \pi} w(\text{edge})$$

**🎯 Example**: Route H → C → F → G costs: 2 + 10 + 3 = 15 minutes

### 🏆 Shortest Path Weight

The **shortest path weight** from vertex $s$ to vertex $t$ is:
$\delta(s,t) = \inf\{w(\pi) \mid \text{path } \pi \text{ from } s \text{ to } t\}$

**🤔 Why "infimum" instead of "minimum"?** Sometimes no finite-length minimum-weight path exists! This happens when negative cycles are involved.

Think: *"What's the cheapest way to get from here to there?"*

### 🔥 Properties That Still Hold (Thankfully!)

#### ✨ Optimal Substructure
If the best route from NYC to LA goes through Chicago, then:
- NYC → Chicago must be the best route to Chicago
- Chicago → LA must be the best route from Chicago to LA

*Why?* If either piece wasn't optimal, you could swap it for something better!

#### 🚫 When There's No Path
$\delta(s,t) = \infty$ (You literally can't get there from here)

### 😱 The Dark Side: Negative Cycles

Here's where things get spicy! 🌶️

#### What's a Negative Cycle?
A loop where going around actually **saves** you money/time/cost:

**💸 Real Example**: Currency arbitrage
- USD → EUR: lose 2%
- EUR → GBP: lose 1%  
- GBP → USD: gain 5%
- Net effect: +2% (free money glitch!)

**🎮 Game Example**: 
- Magic portal costs 10 energy to use
- But lands you next to a fountain that gives 15 energy
- Net: +5 energy per loop!

#### 💥 The Problem
If you can reach a negative cycle, then $\delta(s,t) = -\infty$

*Why?* Keep looping to make your cost arbitrarily negative! This breaks our algorithms unless we're careful.

**🔍 Negative Cycle Detection**: Sometimes we WANT to find these cycles (arbitrage opportunities, infinite energy glitches, etc.)

---

## 🗺️ Your Shortest Path Toolkit {#algorithms-overview}

Think of this as your Swiss Army knife for different graph situations:

### 🛠️ The Complete Algorithm Menu

| 🏗️ Graph Type | ⚖️ Weight Rules | 🧠 Algorithm | ⏱️ Time | 📚 Difficulty |
|---------------|----------------|-------------|---------|-------------|
| Any | No weights | **BFS** | $O(\|V\| + \|E\|)$ | 😊 Easy |
| **DAG** | Any weights | **DAG Relaxation** | $O(\|V\| + \|E\|)$ | 😐 Medium |
| Any | Any weights | **Bellman-Ford** | $O(\|V\| \cdot \|E\|)$ | 😓 Hard |
| Any | Non-negative | **Dijkstra** | $O(\|V\| \log \|V\| + \|E\|)$ | 😤 Tricky |

### 🎯 When Can You Still Use BFS?

BFS is like using a simple calculator—great for basic math:

✅ **Perfect for BFS**:
- All weights are the same (like unweighted)
- All weights positive, small total weight

❌ **BFS Breaks Down**:
- Negative weights (BFS can't "undo" bad decisions)
- Huge weight differences (BFS treats highway = sidewalk)

### 🤯 The General Problem Difficulty

For arbitrary weighted graphs, we **don't know** how to beat $O(|V| \log |V| + |E|)$ time.

**But**: DAGs are special! No cycles = linear time possible! 🎉

### 📚 What's Coming in the Next Four Lectures

This is just the beginning of our weighted shortest paths journey:

- **(L11)**: DAG Relaxation - Perfect for directed acyclic graphs
- **L12**: Bellman-Ford - Handles any weights, detects negative cycles  
- **L13**: Dijkstra - Lightning fast for non-negative weights
- **L14**: Advanced techniques and applications

**🎯 Focus Note**: These lectures focus on computing shortest-path **weights** (distances). We can reconstruct the actual **paths** afterward using the parent pointer technique!

---

## 🌳 Building Your Path Tree {#shortest-paths-tree}

### 🧠 The Smart Separation Strategy

Instead of computing everything at once, we split the problem:

1. **Phase 1**: Find all shortest distances $\delta(s, v)$
2. **Phase 2**: Reconstruct the actual paths

This is like getting all flight prices first, then booking your specific route.

### 🔧 Parent Pointer Construction Algorithm

**The Goal**: Build a tree showing optimal paths from source $s$.

**🎯 Key Insight**: We only need parent pointers for vertices $v$ with finite $\delta(s, v)$ (reachable vertices).

```python
def build_shortest_path_tree(distances, graph):
    parent = {}
    parent[s] = None  # Source has no parent
    
    # Phase 1: Basic parent assignment
    for u in vertices:
        if distances[u] != infinity:  # Only finite distances
            for v in neighbors(u):
                # Is this edge part of a shortest path?
                if (v not in parent and 
                    distances[v] == distances[u] + weight(u, v)):
                    parent[v] = u  # Found shortest path through (u,v)
    
    return parent
```

### ⚠️ The Zero-Weight Cycle Challenge

**Problem**: Parent pointers may traverse cycles of zero weight!

**Example**: A → B (cost 2) → C (cost -1) → A (cost -1) = total cost 0

**Solution Strategy**: 
1. **Mark vertices** in zero-weight cycles
2. **Break cycles** strategically by reassigning parent pointers
3. **Maintain tree structure** while preserving shortest paths

```python
# Phase 2: Handle zero-weight cycles
def handle_zero_weight_cycles(parent, distances, graph):
    marked = set()  # Vertices in zero-weight cycles
    
    # Identify and mark cycles
    for u in vertices:
        if distances[u] != infinity:
            for v in neighbors(u):
                if (v in marked and 
                    distances[v] == distances[u] + weight(u, v)):
                    # Break the cycle by updating parent
                    unmark_cycle_vertices(v, parent, marked)
                    parent[v] = u
```

**🎯 Exercise from MIT**: Prove this algorithm correctly computes parent pointers in $O(|V| + |E|)$ time.

### 💡 Why This Approach is Genius

- **Modularity**: Solve distance problem once, reuse for many path queries
- **Flexibility**: Want path to specific vertex? Just follow parent pointers backwards
- **Efficiency**: Linear time reconstruction vs. exponential path enumeration

---

## 🎯 DAG Relaxation: The Magic Algorithm {#dag-relaxation}

### 🧠 The Big Idea: Relaxation

Think of relaxation like updating your GPS when you find a better route:

#### 📱 Your Mental Model
- **Current estimate** $d(s, v)$: "Best route I know so far to v"
- **Discovery**: "Wait, going through u is cheaper!"
- **Update**: $d(s, v) = d(s, u) + w(u, v)$

#### 📐 The Triangle Inequality (Your Mathematical GPS)

For any three places u, x, v:
$$\delta(u, v) \leq \delta(u, x) + \delta(x, v)$$

**Translation**: The direct route can't be longer than any detour route.

**🎯 Violation Check**: If $d(s, v) > d(s, u) + w(u, v)$, we found a better path!

### 🔧 The DAG Relaxation Algorithm

```python
def dag_shortest_paths(graph, weights, source):
    # Step 1: Initialize distances
    distance = {v: float('inf') for v in vertices}
    distance[source] = 0
    
    # Step 2: Get topological ordering
    topo_order = topological_sort(graph)
    
    # Step 3: Process vertices in topological order
    for u in topo_order:
        for v in neighbors(u):
            # Relaxation step: try to improve path to v
            if distance[v] > distance[u] + weights[(u, v)]:
                distance[v] = distance[u] + weights[(u, v)]
                # "Hey v, I found you a better deal!"
    
    return distance
```

### 🎮 Walkthrough Example: The Quest Game

Imagine a quest game where you move through locations:

**Map**: Start → Village → Forest → Castle → Treasure
**Costs**: Various energy costs for each path

**🎬 The Movie**:
1. **Frame 1**: Start with distance[Start] = 0, everything else = ∞
2. **Frame 2**: Process Start → update Village's distance
3. **Frame 3**: Process Village → update Forest and maybe Castle
4. **Frame 4**: Process Forest → update Castle and Treasure
5. **Frame 5**: Process Castle → update Treasure (if better)

### 🤔 Why Topological Order is Crucial

**The Rule**: Process vertex u before any vertex u can reach.

**Why it works**: When we process vertex v, we've already found optimal paths to ALL vertices that can reach v. So we can safely finalize v's distance.

**Breaking the rule**: If we process vertices randomly, we might miss better paths that get discovered later!

---

## ✅ Why It Actually Works (Correctness Proof) {#correctness}

### 🎯 The Main Claim

**Big Promise**: After DAG Relaxation finishes, $d(s, v) = \delta(s, v)$ for every vertex $v$.

Translation: *"We find the TRUE shortest distance to every vertex."*

### 🧩 Proof Strategy: Mathematical Induction

Think of this like proving dominoes fall: if the first few work, and each success causes the next success, then they all work!

#### 🏁 Base Case
The source vertex $s$ starts with $d(s, s) = 0 = \delta(s, s)$. ✅

Any unreachable vertices stay at $\infty$, which is correct. ✅

#### 🔄 Inductive Step

**Assume**: First $k$ vertices in topological order have correct distances.

**Prove**: The $(k+1)$-th vertex $v$ gets the correct distance.

**🎯 Key Insight**: Consider the TRUE shortest path to $v$:
```
s → ... → u → v
```

Since it's a DAG, vertex $u$ comes before $v$ in topological order.

**The Logic Chain**:
1. By induction: $d(s, u) = \delta(s, u)$ ✅
2. When processing $u$: we relax edge $(u, v)$
3. This sets: $d(s, v) \leq \delta(s, u) + w(u, v) = \delta(s, v)$
4. But relaxation safety ensures: $d(s, v) \geq \delta(s, v)$
5. Therefore: $d(s, v) = \delta(s, v)$ ✅

### 🎪 Alternative Proof (The Elegant Version)

**Direct reasoning**: For any vertex $v$, DAG relaxation effectively computes:
$d(s, v) = \min\{d(s, u) + w(u, v) \mid u \in \text{Adj}^-(v)\}$

Where $\text{Adj}^-(v)$ means all vertices that have edges pointing TO $v$.

**🧠 Why this works**:
- Every shortest path to $v$ must pass through some incoming neighbor $u$ of $v$
- By induction: $d(s, u) = \delta(s, u)$ for all such neighbors $u$  
- Therefore: $d(s, v) = \delta(s, v)$ ✅

**🎯 Intuition**: It's like asking all your friends "What's the cheapest way to reach v through you?" and taking the best offer.

---

## ⏱️ Speed Analysis: Breaking Down the Performance {#running-time}

### 🔍 Component-by-Component Analysis

#### 1️⃣ Initialization: $O(|V|)$
```python
# Set all distances to infinity, source to 0
for v in vertices:
    distance[v] = float('inf')  # O(1) per vertex
distance[source] = 0            # O(1)
```
**Total**: $|V|$ vertices × $O(1)$ = $O(|V|)$

#### 2️⃣ Topological Sort: $O(|V| + |E|)$
Using DFS-based topological sort:
- Visit each vertex once: $O(|V|)$
- Explore each edge once: $O(|E|)$
- **Total**: $O(|V| + |E|)$

#### 3️⃣ Relaxation Loop: $O(|E|)$
```python
for u in topological_order:          # |V| iterations
    for v in neighbors(u):           # deg⁺(u) iterations
        # Relaxation: O(1)
```

**Key insight**: Each edge gets relaxed exactly once!

**🧮 The Math**: 
$\sum_{u \in V} \deg^+(u) = |E|$

This is because every edge $(u,v)$ contributes exactly 1 to $\deg^+(u)$, and there are $|E|$ total edges.

So the **additional work** is upper bounded by $O(1) \times |E| = O(|E|)$ total time.

### 🏆 Final Running Time

$$O(|V|) + O(|V| + |E|) + O(|E|) = O(|V| + |E|)$$

**🎉 Linear Time!** This is optimal—we have to look at every vertex and edge at least once.

### 💡 Why DAGs Make This Possible

**The Magic**: Topological ordering ensures we process vertices in the perfect order. Each vertex's distance gets finalized in exactly one pass.

**Contrast with Cycles**: If there were cycles, we might need multiple passes:
- Pass 1: Get rough estimates
- Pass 2: Improve estimates using Pass 1 results  
- Pass 3: Improve further...
- Eventually converge (Bellman-Ford does this)

**DAGs shortcut this**: Process in the right order, get correct answers immediately!

---

## 🎯 Key Takeaways & Battle Strategies

### 🧠 Mental Models to Remember

1. **Weighted Graphs = Real World**: Every journey has different costs
2. **Relaxation = GPS Updates**: Always looking for better routes
3. **Negative Cycles = Free Money Glitch**: Breaks normal assumptions
4. **DAGs = Perfect Order**: No cycles means we can solve optimally in one pass
5. **Topological Sort = Dependency Resolution**: Like figuring out which courses to take first

### 🎪 Fun Applications to Try

#### 🎮 Game Development
- **Movement costs**: Different terrain types (swamp vs. road)
- **Resource management**: Energy costs for different actions
- **Quest optimization**: Minimum time to complete storyline

#### 💰 Finance & Economics  
- **Arbitrage detection**: Finding negative cycles in currency exchanges
- **Supply chain optimization**: Minimize cost/time in shipping
- **Investment strategies**: Optimal portfolio rebalancing

#### 🌐 Network Engineering
- **Routing protocols**: How internet packets find efficient paths
- **Load balancing**: Distributing traffic optimally
- **Failure recovery**: Rerouting when links go down

### 🔮 Coming Up Next

- **Bellman-Ford Algorithm**: Handles negative cycles like a boss
- **Dijkstra's Algorithm**: Lightning-fast for non-negative weights
- **Floyd-Warshall**: All-pairs shortest paths (the nuclear option)

### 🎯 Pro Tips for Implementation

1. **Always check for negative cycles** in real applications
2. **Use the right algorithm** for your weight constraints
3. **Consider preprocessing** if you'll make many queries
4. **Profile your code**—theory vs. practice can differ!

---

## 🚀 Beyond the Basics: Advanced Insights

### 🔥 When Theory Meets Practice

**Memory Access Patterns**: In real implementations, cache performance matters more than you think!

**Parallel Processing**: DAGs are naturally parallelizable—process independent vertices simultaneously!

**Dynamic Updates**: What happens when edge weights change? Sometimes you can incrementally update instead of recomputing from scratch.

### 🎭 The Philosophy of Algorithm Design

DAG Relaxation embodies beautiful principles:
- **Divide and Conquer**: Break complex problems into simpler pieces
- **Optimal Substructure**: Use solutions to subproblems optimally  
- **Greedy Choice**: Make locally optimal decisions that lead to global optimality
- **Mathematical Elegance**: Simple rules that provably work

---

> *"The shortest path between two truths in the real world passes through the world of action."* - Alfred North Whitehead (adapted for Computer Science)

**Remember**: Algorithms aren't just about solving problems—they're about understanding the structure of problems so deeply that solutions become inevitable.

---

*These notes are based on MIT 6.006 Introduction to Algorithms, Spring 2020*  
*Enhanced with examples, intuition, and real-world connections*  
*Available at: https://ocw.mit.edu*

**📋 MIT OpenCourseWare Terms**: For information about citing these materials or terms of use, visit: https://ocw.mit.edu/terms